In [1]:
import pandas as pd

# Read data into DataFrame
df = pd.read_csv('data/a1Yields.csv')

df.head()

,Run,Angle,Detector,Ep,Q int,Yield,Yield effcor,Yield err,Yield err effcor
0,97,120,0,2499.3,1140000.0,0.000574,3.050882e-11,0.000044,2.358194e-12
1,97,105,1,2499.3,1140000.0,0.000581,3.273932e-11,0.000053,2.962391e-12
2,97,90,2,2499.3,1140000.0,0.000308,2.942379e-11,0.000045,4.336213e-12
3,97,45,3,2499.3,1140000.0,0.000651,3.722398e-11,0.000061,3.500768e-12
4,97,30,4,2499.3,1140000.0,0.000627,3.469573e-11,0.000059,3.263903e-12


# Create sql database manually using 'pandas'

In [2]:
import sqlalchemy
sqlalchemy.__version__ 

'1.3.13'

In [3]:
from sqlalchemy import create_engine

# Create SQLite database (relative path)
engine = create_engine('sqlite:///sqlalchemy_yields.db', echo=True)

# Raise exception if it exists
# df.to_sql('a1', con=engine, if_exists='fail', index=True)

# Drops existing table before inserting new values
df.to_sql('a1', con=engine, if_exists='replace', index=True)

2020-03-05 01:17:28,314 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-03-05 01:17:28,317 INFO sqlalchemy.engine.base.Engine ()
2020-03-05 01:17:28,318 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-03-05 01:17:28,319 INFO sqlalchemy.engine.base.Engine ()
2020-03-05 01:17:28,320 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("a1")
2020-03-05 01:17:28,321 INFO sqlalchemy.engine.base.Engine ()
2020-03-05 01:17:28,322 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("a1")
2020-03-05 01:17:28,322 INFO sqlalchemy.engine.base.Engine ()
2020-03-05 01:17:28,325 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE a1 (
	"index" BIGINT, 
	"Run" BIGINT, 
	"Angle" BIGINT, 
	"Detector" BIGINT, 
	"Ep" FLOAT, 
	"Q int" FLOAT, 
	"Yield" FLOAT, 
	"Yield effcor" FLOAT, 
	"Yield err" FLOAT, 
	"Yield err effcor" FLOAT
)


2020-03-05 01:17:28,325 INFO sqlalchemy.engine.base.Engine ()
20

# Create sql database manually using 'sqlite3'

https://docs.python.org/3.8/library/sqlite3.html

In [4]:
import sqlite3
print(sqlite3.version)

# Create connection object (represent database)
conn = sqlite3.connect('sqlite_yields.db')

# Create cursor object
cur = conn.cursor()

2.6.0


In [5]:
# Drop table
# cur.execute('''DROP TABLE a1''')

# Create table
cur.execute('''CREATE TABLE a1 
            (Run INTEGER, Angle INTEGER, Detector INTEGER, 
            Ep REAL, Q_int INTEGER, Yield REAL, 
            Yield_effcor REAL, Yield_err REAL, Yield_err_effcor REAL)''')

### Use execute to insert single values/row and use executemany for multiple values/rows

In [6]:
column_names = []
for names in cur.execute("SELECT * FROM a1").description:
    column_names.append(names[0])
print("Columns in SQL database:",column_names)


# Insert values stored in DataFrame into the database
# iterrows() returns iterable (tuple of row_index,row_values)
cur.executemany("INSERT INTO a1 (Run,Angle,Detector,Ep,Q_int,Yield,Yield_effcor,Yield_err,Yield_err_effcor) VALUES (?,?,?,?,?,?,?,?,?)", [(x[1]) for x in df.iterrows()] )



# Store values of an array/series object into database
# HUGE WARNING: subsequent executes start from last index (append to final row in database)
# cur.executemany("INSERT INTO a1 (Angle) VALUES (?)", [(x[1],) for x in df['Angle'].iteritems()] )


# Save changes
conn.commit()

Columns in SQL database: ['Run', 'Angle', 'Detector', 'Ep', 'Q_int', 'Yield', 'Yield_effcor', 'Yield_err', 'Yield_err_effcor']


### Grab rows from database

In [7]:
cur.execute("SELECT * FROM a1")

# data = cur.fetchone()
data = cur.fetchmany(5)
# data = cur.fetchall()

for row in data[:5]:
    print(row)

(97, 120, 0, 2499.3, 1140000, 0.0005744190000000001, 3.050881567640499e-11, 4.44e-05, 2.35819395951802e-12)
(97, 105, 1, 2499.3, 1140000, 0.0005813169999999999, 3.273931968860119e-11, 5.260000000000001e-05, 2.9623909426705622e-12)
(97, 90, 2, 2499.3, 1140000, 0.000308066, 2.942378714934269e-11, 4.54e-05, 4.336213462635143e-12)
(97, 45, 3, 2499.3, 1140000, 0.000650745, 3.7223979790045365e-11, 6.12e-05, 3.5007684471655963e-12)
(97, 30, 4, 2499.3, 1140000, 0.000627178, 3.4695728929745706e-11, 5.9e-05, 3.263902762620813e-12)


### Grab a column

In [8]:
conn.row_factory = sqlite3.Row

cur = conn.cursor()
cur.execute("SELECT * FROM a1")

# colData = cur.fetchone()
colData = cur.fetchmany(5)
# colData = cur.fetchall()

print(colData[0].keys(),'\n\n')

angleList = []
for row in colData[:5]:
    angleList.append(row['Angle'])
    print(row['Angle'])

['Run', 'Angle', 'Detector', 'Ep', 'Q_int', 'Yield', 'Yield_effcor', 'Yield_err', 'Yield_err_effcor'] 


120
105
90
45
30


### Alter values in a column

# Close out connections

In [9]:
# cur.close()
# conn.close()